In [ ]:
!pip install -q -r requirements.txt
import numpy as np
import os
import tensorflow as tf
from functools import partial
from preprocessing import LABELS
from time import time
import random
from preprocessing import get_mfccs
from functools import partial

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
2022-12-15 09:30:58.498073: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 09:30:58.610317: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-15 09:30:58.615485: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-15 09:

In [ ]:
train_ds = tf.data.Dataset.list_files(['./msc-test/go*','./msc-test/stop*' ])
test_ds = tf.data.Dataset.list_files(['./msc-test/go*','./msc-test/stop*' ])
val_ds = tf.data.Dataset.list_files(['./msc-val/go*','./msc-val/stop*' ])

2022-12-15 09:31:00.147914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-15 09:31:00.147943: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-15 09:31:00.147958: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-c40ae498-620e-4b09-a576-182e0c5bf3b3): /proc/driver/nvidia/version does not exist
2022-12-15 09:31:00.148219: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Deterministic environment 
seed = 17
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
TRAINING_ARGS = {
    'batch_size': 20,
    'epochs': 60,
    'initial_learning_rate': 0.001,
    'end_learning_rate': 1.e-5,
}

In [ ]:
MFCCS_ARGS = {
    'downsampling_rate': 16000,
    'frame_length_in_s': 0.032,
    'frame_step_in_s': 0.016,
    'num_mel_bins': 10,
    'lower_frequency': 20,
    'upper_frequency': 8000,
    'num_coefficients': 10
}

In [ ]:
# Setting Preprocessing function
get_mfccs_frozen = partial(get_mfccs, **MFCCS_ARGS)

def resize_mfccs_(filename):
    signal, label = get_mfccs_frozen(filename)
    signal = tf.expand_dims(signal, -1)
    #signal = tf.image.resize(signal, [32, 32])
    label_id = tf.argmax(label == LABELS)
    return signal, label_id

In [ ]:
# Applying it to DS
train_ds_ = train_ds.map(resize_mfccs_).batch(TRAINING_ARGS['batch_size']).cache()
test_ds_ = test_ds.map(resize_mfccs_).batch(TRAINING_ARGS['batch_size'])
val_ds_ = val_ds.map(resize_mfccs_).batch(TRAINING_ARGS['batch_size'])

2022-12-15 09:31:00.610585: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-12-15 09:31:00.610787: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 AVX512F FMA
2022-12-15 09:31:01.011348: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-15 09:31:01.012834: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-15 09:31:01.013015: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-15 09:31:01.382170: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at functional_ops.cc:373 : INTERNAL: No function library
2022-12-15 09:31:01.383606: W tensorflow/core/framework/op_kernel.cc:1780] O

In [ ]:
for example_tensor,example_label in train_ds_.take(1):
    example_batch = example_tensor
example_batch.shape

2022-12-15 09:31:02.258314: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


TensorShape([20, 61, 10, 1])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=128, kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1],
        use_bias=True, padding='same'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=[1, 1], strides=[1, 1],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dense(units=len(LABELS)),
    tf.keras.layers.Softmax()
])
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

initial_learning_rate = TRAINING_ARGS['initial_learning_rate']
end_learning_rate = TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=len(train_ds_) * TRAINING_ARGS['epochs'],
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 4, 128)        1152      
                                                                 
 batch_normalization (BatchN  (None, 30, 4, 128)       512       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 30, 4, 128)        0         
                                                                 
 dropout (Dropout)           (None, 30, 4, 128)        0         
                                                                 
 depthwise_conv2d (Depthwise  (None, 30, 4, 128)       1280      
 Conv2D)                                                         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 4, 32)         4

In [ ]:
timestamp = time()
history = model.fit(
    train_ds_,
    epochs=TRAINING_ARGS['epochs'],
    validation_data=val_ds_,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ]
)

Epoch 1/60
10/10 [==============================] - 2s 160ms/step - loss: 1.7681 - sparse_categorical_accuracy: 0.5700 - val_loss: 0.6850 - val_sparse_categorical_accuracy: 0.5050
Epoch 2/60
10/10 [==============================] - 1s 113ms/step - loss: 0.6016 - sparse_categorical_accuracy: 0.8000 - val_loss: 0.6745 - val_sparse_categorical_accuracy: 0.5950
Epoch 3/60
10/10 [==============================] - 1s 109ms/step - loss: 0.3519 - sparse_categorical_accuracy: 0.8550 - val_loss: 0.6631 - val_sparse_categorical_accuracy: 0.7650
Epoch 4/60
10/10 [==============================] - 1s 109ms/step - loss: 0.2579 - sparse_categorical_accuracy: 0.9100 - val_loss: 0.6530 - val_sparse_categorical_accuracy: 0.8250
Epoch 5/60
10/10 [==============================] - 1s 110ms/step - loss: 0.2173 - sparse_categorical_accuracy: 0.9200 - val_loss: 0.6436 - val_sparse_categorical_accuracy: 0.8200
Epoch 6/60
10/10 [==============================] - 1s 110ms/step - loss: 0.1674 - sparse_categorica

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds_)

10/10 [==============================] - 1s 45ms/step - loss: 0.0203 - sparse_categorical_accuracy: 1.0000


In [ ]:
test_accuracy = test_accuracy
training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]
print(f'Training loss: {training_loss:.3f}')
print(f'Training accuracy: {training_accuracy * 100:.2f}%')
print(f'Validation loss: {val_loss:.3f}')
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.3f}')
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

Training loss: 0.014
Training accuracy: 100.00%
Validation loss: 0.183
Validation accuracy: 95.50%
Test loss: 0.020
Test accuracy: 100.00%


In [ ]:
import zipfile
import tempfile


def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in kilobytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)/1000
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
model_file = '/work/tflite_models/15_dec_2022.tflite'
# Save the model.
with open(model_file, 'wb') as f:
    f.write(tflite_model)

print("Model Size: ", get_gzipped_model_size(model_file), ' KB')
print("Model Size (no compression): ", os.path.getsize(model_file)/1000, ' KB')

INFO:tensorflow:Assets written to: /tmp/tmph1_yz0_f/assets
INFO:tensorflow:Assets written to: /tmp/tmph1_yz0_f/assets
Model Size:  10.625  KB
Model Size (no compression):  14.56  KB
2022-12-15 09:32:04.524060: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-12-15 09:32:04.524107: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-12-15 09:32:04.524826: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmph1_yz0_f
2022-12-15 09:32:04.527023: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-12-15 09:32:04.527047: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmph1_yz0_f
2022-12-15 09:32:04.532171: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-12-15 09:32:04.533661: I tensorflow/cc/saved_model/loader.cc:229